In [ ]:
# ASD Therapy scheduling optimization using PSO algorithm with Adaptive Reinforcement Double DQN techniques
import random
import numpy as np
import scipy.special
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import defaultdict


class Patient:
    def __init__(self, id):
        self.id = id


class Provider:
    def __init__(self, id):
        self.id = id


class Facility:
    def __init__(self, id):
        self.id = id


class Appointment:
    def __init__(self):
        self._patient = None
        self._provider = None
        self._facility = None
        self._day = int
        self._slot = int

    @property
    def patient(self):
        return self._patient

    @patient.setter
    def patient(self, value):
        if isinstance(value, Patient):
            self._patient = value
        else:
            raise TypeError("Patient must be an instance of Patient class")

    @property
    def provider(self):
        return self._provider

    @provider.setter
    def provider(self, value):
        if isinstance(value, Provider):
            self._provider = value
        else:
            raise TypeError("Provider must be an instance of Provider class")

    @property
    def facility(self):
        return self._facility

    @facility.setter
    def facility(self, value):
        if isinstance(value, Facility):
            self._facility = value
        else:
            raise TypeError("Facility must be an instance of Facility class")

    @property
    def day(self):
        return self._day

    @day.setter
    def day(self, value):
        self._day = value

    @property
    def slot(self):
        return self._slot

    @slot.setter
    def slot(self, value):
        self._slot = value

    @classmethod
    def get_appointment_list(cls, data):
        appointments = []
        for row in data:
            appointment = cls()
            appointment.patient = Patient(row[0])
            appointment.provider = Provider(row[1])
            appointment.facility = Facility(row[2])
            appointment.day = row[3]
            appointment.slot = row[4]
            appointments.append(appointment)
        return appointments


class ScheduleGenerator:
    def __init__(self, num_patients, num_providers, num_facilities, num_days, slots_per_day):
        self.num_patients = num_patients
        self.num_providers = num_providers
        self.num_facilities = num_facilities
        self.num_days = num_days
        self.slots_per_day = slots_per_day

    def generate_schedule(self):
        schedule = []
        for patient_id in range(self.num_patients):
            provider_id = random.randint(0, self.num_providers - 1)
            facility_id = random.randint(0, self.num_facilities - 1)
            day = random.randint(0, self.num_days - 1)
            slot = random.randint(0, self.slots_per_day - 1)
            schedule.append([patient_id, provider_id, facility_id, day, slot])
        return np.array(schedule)


class FitnessCalculator:
    def __init__(self, schedule):
        self.appointments = Appointment.get_appointment_list(schedule)

    def _calculate_minimize_therapy_duration(self):
        """
            Calculates the fitness of a schedule by minimizing therapy duration.

            Args:
                self.appointments: A list of tuples, where each tuple represents a scheduled appointment
                          and contains (patient, provider, facility, day, slot).

            Returns:
                The fitness score (lower is better), representing the total duration
                of therapy for all patients.
            """
        patient_therapy_days = {}
        for appointment in self.appointments:
            patient_id = appointment.patient.id
            day = appointment.day
            if patient_id not in patient_therapy_days:
                patient_therapy_days[patient_id] = set()
            patient_therapy_days[patient_id].add(day)

        total_duration = sum([len(days) for days in patient_therapy_days.values()])
        return total_duration

    def _calculate_maximize_provider_utilization(self):
        """
            Calculates the fitness of a schedule by maximizing provider utilization.

            Args:
                self.appointments: A list of tuples, where each tuple represents a scheduled appointment
                          and contains (patient, provider, facility, day, slot).

            Returns:
                The fitness score (higher is better), representing the total number of appointments scheduled for all providers.
            """
        provider_utilization = {}
        for appointment in self.appointments:
            provider_id = appointment.provider.id
            provider_utilization[provider_id] = provider_utilization.get(provider_id, 0) + 1

        # Calculate MAX_APPOINTMENTS_PER_PROVIDER
        max_appointments_per_provider = 0
        for provider_id, appointments in provider_utilization.items():
            max_appointments_per_provider = max(max_appointments_per_provider, appointments)

        total_utilization = sum(provider_utilization.values())

        # Add a penalty if providers are overworked
        overworked_penalty = 0
        for provider_id, appointments in provider_utilization.items():
            if appointments > max_appointments_per_provider:
                overworked_penalty += appointments - max_appointments_per_provider

        fitness = total_utilization - overworked_penalty
        return fitness

    def calculate_fitness(self, weights):
        # Calculate individual objective function values
        f1 = self._calculate_minimize_therapy_duration()
        f2 = self._calculate_maximize_provider_utilization()
        f3 = -self._calculate_constraint_violation()

        # Weighted sum
        fitness = weights[3] * f1 + weights[4] * f2 + weights[5] * f3

        return fitness

    def _calculate_constraint_violation(self):
        violation_score = 0

        # 1. Provider availability: Ensure no overlapping appointments for a provider.
        provider_appointments = {}
        for appointment in self.appointments:
            provider_id = appointment.provider.id
            if provider_id not in provider_appointments:
                provider_appointments[provider_id] = []
            provider_appointments[provider_id].append((appointment.day, appointment.slot))

        for provider_id, appointments in provider_appointments.items():
            appointments.sort(key=lambda x: (x[0], x[1]))
            for i in range(len(appointments) - 1):
                if appointments[i] == appointments[i + 1]:
                    violation_score += 1  # Overlapping appointments

        # 2. Checks if there are overlapping appointments within the same facility, day, and slot for the same provider.
        facility_appointments = defaultdict(list)
        for appointment in self.appointments:
            facility_id = appointment.facility.id
            provider_id = appointment.provider.id
            if (facility_id, appointment.day, appointment.slot, provider_id) in facility_appointments:
                violation_score += 1  # Overlapping appointment found
            facility_appointments[(facility_id, appointment.day, appointment.slot, provider_id)].append(appointment)

        return violation_score


# Deep Q-Network (DQN)
class DQN:
    def __init__(self, state_size, num_weights):
        self.state_size = state_size
        self.num_weights = num_weights
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.gamma = 0.95  # Discount rate
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())

    def _build_model(self):
        model = keras.Sequential([
            keras.layers.Dense(24, input_dim=self.state_size, activation='leaky_relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(24, activation='leaky_relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(self.num_weights, activation='linear')
        ])
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def predict(self, state):
        return self.model.predict(state)

    def train(self, state, action, reward, next_state, done):
        target = reward

        if not done:
            # Use target network to select action
            target_actions = self.target_model.predict(next_state)
            best_action = np.argmax(target_actions[0])

            # Use main network to estimate Q-value for the selected action
            target_q_value = self.model.predict(next_state)[0][best_action]
            target = reward + self.gamma * target_q_value

        target_f = self.model.predict(state)
        # Convert action to an integer index
        action_index = np.argmax(action)  # Selecting the index of the highest weight
        target_f[0][action_index] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)

        # Update target network periodically
        if np.random.rand() < 0.1:
            self.target_model.set_weights(self.model.get_weights())


#  Reinforcement learning
class RL_Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.dqn = DQN(state_size, action_size)
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def take_action(self, state):
        # Clip the state to prevent numerical issues
        clipped_state = np.clip(state, -1, 1)
        if np.random.rand() <= self.epsilon:
            return np.random.rand(self.action_size)
        act_values = self.dqn.predict(clipped_state)
        # Check for NaN values and handle appropriately
        if np.isnan(act_values).any():
            return np.random.rand(self.action_size)
        return act_values[0]

    def learn(self, state, action, reward, next_state, done):
        self.dqn.train(state, action, reward, next_state, done)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


class Helper:
    @staticmethod
    def calculate_state(iteration, global_best_fitness, average_fitness, std_fitness):
        """Calculates the state for the RL agent."""
        return np.array([iteration, global_best_fitness, average_fitness, std_fitness]).reshape(1, -1)

    @staticmethod
    def calculate_reward(current_fitness, previous_fitness):
        """Calculates the reward for the RL agent."""
        return current_fitness - previous_fitness


class ConvergencePlotter:
    def __init__(self, fitness_history, title="Convergence Plot"):
        self.fitness_history = fitness_history
        self.title = title
        self.x_label = "Iteration"
        self.y_label = "Fitness"

    def plot(self):
        plt.plot(range(len(self.fitness_history)), self.fitness_history)
        plt.xlabel(self.x_label)
        plt.ylabel(self.y_label)
        plt.title(self.title)
        plt.grid(True)
        plt.show()


class Particle:
    def __init__(self, position):
        self.position = position
        self.velocity = np.zeros_like(position)
        self.best_position = np.copy(position)
        self.best_fitness = -float('inf')  # Initialize with a very low value

    def update_velocity_and_position(self, global_best_position, w, c1, c2):
        r1 = random.random()
        r2 = random.random()

        # Ensure all elements are iterable, if not, convert them to a list
        velocity = [v if isinstance(v, (list, tuple, np.ndarray)) else [v] for v in self.velocity]
        best_position = [bp if isinstance(bp, (list, tuple, np.ndarray)) else [bp] for bp in self.best_position]
        position = [p if isinstance(p, (list, tuple, np.ndarray)) else [p] for p in self.position]
        global_best_position = [gbp if isinstance(gbp, (list, tuple, np.ndarray)) else [gbp] for gbp in
                                global_best_position]

        # Update velocity component-wise
        for i in range(len(velocity)):
            for j in range(len(velocity[i])):
                new_velocity = (w * velocity[i][j] +
                                c1 * r1 * (best_position[i][j] - position[i][j]) +
                                c2 * r2 * (global_best_position[i][j] - position[i][j]))

                # Avoid potential division by zero
                if abs(best_position[i][j] - position[i][j]) > 1e-8:
                    new_velocity = scipy.special.xlogy(new_velocity, abs(best_position[i][j] - position[i][j]))
                else:
                    new_velocity = 0  # Reset velocity
                new_velocity = np.clip(new_velocity, 0, 1)  # Ensure velocity is within the bounds
                velocity[i][j] = new_velocity

        # Update position
        self.position = [
            tuple(int(x + y) if isinstance(x, (int, float)) else x + y for x, y in zip(p, v))
            for p, v in zip(self.position, self.velocity)
        ]

    def update_best_position(self, weights):
        fitness = FitnessCalculator(self.position).calculate_fitness(weights)
        if fitness > self.best_fitness:
            self.best_fitness = fitness
            self.best_position = np.copy(self.position)


class PSO:
    def __init__(self, num_patients, num_providers, num_facilities, num_days, slots_per_day, num_particles,
                 max_iterations, inertia_weight, cognitive_constant, social_constant):
        self.num_particles = num_particles
        self.max_iterations = max_iterations
        self.inertia_weight = inertia_weight
        self.cognitive_constant = cognitive_constant
        self.social_constant = social_constant
        self.swarm = [Particle(
            ScheduleGenerator(num_patients, num_providers, num_facilities, num_days, slots_per_day).generate_schedule())
            for _ in range(num_particles)]
        self.global_best_position = self.swarm[0].best_position
        self.global_best_fitness = self.swarm[0].best_fitness
        self.previous_best_fitness = self.global_best_fitness
        self.agent = RL_Agent(state_size=4, action_size=6)
        self.helper = Helper()

    def adjust_pso_parameters(self, weights):
        self.inertia_weight = weights[0]
        self.cognitive_constant = weights[1]
        self.social_constant = weights[2]

    def optimize(self):
        fitness_history = []  # Track global best fitness at each iteration
        for iteration in range(self.max_iterations):
            average_fitness = np.mean([particle.best_fitness for particle in self.swarm])
            std_fitness = np.std([particle.best_fitness for particle in self.swarm])
            state = self.helper.calculate_state(iteration, self.global_best_fitness, average_fitness, std_fitness)

            weights = self.agent.take_action(state)
            self.adjust_pso_parameters(weights)

            for particle in self.swarm:
                particle.update_velocity_and_position(self.global_best_position, self.inertia_weight,
                                                      self.cognitive_constant, self.social_constant)
                particle.update_best_position(weights)

            for particle in self.swarm:
                if particle.best_fitness > self.global_best_fitness:
                    self.global_best_position = np.copy(particle.best_position)
                    self.global_best_fitness = particle.best_fitness

            # Calculate next state and reward
            next_state = self.helper.calculate_state(iteration + 1, self.global_best_fitness, average_fitness,
                                                     std_fitness)
            reward = self.helper.calculate_reward(self.global_best_fitness, self.previous_best_fitness)

            # Train the agent
            self.agent.learn(state, weights, reward, next_state, False)
            self.previous_best_fitness = self.global_best_fitness
            fitness_history.append(self.global_best_fitness)

        plotter = ConvergencePlotter(fitness_history)
        plotter.plot()

        return self.global_best_position, self.global_best_fitness


if __name__ == "__main__":
    # Constants
    NUM_PATIENTS = 36
    NUM_PROVIDERS = 3
    NUM_FACILITIES = 2
    NUM_DAYS = 2  # Number of days in a week for scheduling
    NUM_SLOTS_PER_DAY = 3  # Maximum patients a provider can see per day

    # PSO parameters
    NUM_PARTICLES = 50
    MAX_ITERATIONS = 100
    INERTIA_WEIGHT = 0.5
    COGNITIVE_CONSTANT = 1
    SOCIAL_CONSTANT = 2

    pso = PSO(NUM_PATIENTS, NUM_PROVIDERS, NUM_FACILITIES, NUM_DAYS, NUM_SLOTS_PER_DAY, NUM_PARTICLES, MAX_ITERATIONS,
              INERTIA_WEIGHT, COGNITIVE_CONSTANT, SOCIAL_CONSTANT)
    best_schedule, best_fitness = pso.optimize()

    print("Best schedule:", best_schedule.astype(int))
    print("Best fitness:", best_fitness)

# Convert the schedule to a pandas DataFrame for better visualization
df_schedule = pd.DataFrame(best_schedule.astype(int), columns=['Patient', 'Provider', 'Facility', 'Day', 'Slot'])
df_schedule
